<img src="assets/logos/Logo_endpoint.png" width=500></img>


## Code
code is in /containers and each directory there containes the following files
    
- ### /containers/*/Makefile
Makefile for creating the tar.gz file of code for sagemaker framework containers

- ### /containers/*/host.py
The code for generating inferences

- ### /containers/*/test.py
code for testing the the host.py and train.py files localy

- ### /containers/*/train.py
the training code


## Deployment flow
<img src="assets/build/deployment-flow.png" width=800></img>



## Pipeline Config
You pipeline is using aws-sagemaker-build which you should read more about [here](https://aws.amazon.com/blogs/machine-learning/automated-and-continuous-deployment-of-amazon-sagemaker-models-with-aws-step-functions/)

- deployment configuration (hyperparamters, HPO configuration, deployment environmental variables) is part of the cloudformation template in __/templates/main/sagebuild/shoot.js__ or __/templates/main/sagebuild/layout.js__
    - this is a nodejs file that is used to generate the cloudformation template for you infastructure. you should not need to edit it more than just replace value in the apporiate object
- LayoutPipeline and ShootPipeline objects configure the pipelines for the respective endpoint. The follow describes the properties you change/should change

|Parameter Name| Description | Suggested Value|
|---|---|---|
|ConfigFramework"|the framework you want to use, one of "TENSORFLOW" "MXNET" or "SCIKIT"|"MXNET"|
|frameworkversion|frame work version|"1.3.0"|
|inputmode|"File" or "Pipe" depending on what input mode you want|"File"|
|hyperparameters|key value hyperparameters to send to your training job|{}|
|channels|the data channels for the instance, the suggested value will create a training data channel pointing to data with the 'data/train' prefix in the data bucket|{"train":{"path":"data/train"}}|
|trainvolumesize|how much EBS storage to attach to the training instances|"5"|
|traininstancecount|how many instances you want to use for training|1|
|hostinstancetype|inference instance type|"ml.t2.medium"|
|traininstancetype|training instance type|"ml.m5.large"|

for for details on possible parameters and values see the [aws-sagemaker-build documentation](https://github.com/aws-samples/aws-sagemaker-build#Parameters). After you make changes run the following to apply them to your development pipeline
```shell
npm run update
```

## Test

### Local
- edit /containers/*/test.py to configure your testing
- run test.py
```shell
./test.py
```

### Deployed
- train and deploy your model

```shell
npm run up # create testing infastructure
./bin/startLayoutPipeline.js # start deployment for Defensive team
./bin/startShootPipeline.js # start deployment for Offensive team

```
- edit /bin/testLayout.py or /bin/testShoot.py
- run 
```shell
/bin/testLayout.py # test for Defensive team
/bin/testShoot.py # test for Offensive team
```

## Pushing your changes
once you have made your changes and are ready to deploy them to production follow these steps
1. run the following command to stage files you change to be commited
```shell
git add <your file>
```
1. commit your changes with a message
```shell
git commit -m "<your message>"
```
1. push your changes to the remote repository
```shell
git push shoot #for offense
git push layout # for defense
```
1. inform your opperations that there are changes to deploy

## Data
All games are recorded and available in the game archive bucket. You will not be able to access the the data bucket until the opperations group has registered your team. 

### Download
The following script will download all the data from the S3 bucket (set in the config.js), unzip it, and merge it into a single file at /data/data.json
```shell
./bin/download_data.sh
```

### Format:
The /data/data.json file is a new line deliminated json file with the following format
```json
{                                                                                      
    "Status":"Finished",
    "winner":"B",
    "start_time":"<epoch time>",
    "end_time":"<epoch time>",
	"GameId":"d0d73e64-bd47-4bb2-ab50-ed0bd1935066",
    "TeamA":{
        "layout":[[0,0 ...],...],
        "score":0,
        "session":{},
        "name":"purple people eaters",
		"shots":[{"x":6,"y":1},....]
     },
     "TeamB":{
        "layout":[[0,0 ...],...],
        "score":0,
        "session":{},
        "name":"the dr strange loves",
		"shots":[{"x":1,"y":4},....]
	 }
}
```
|top level field|Description|
|---|---|
|Status||
|winner||
|start_time||
|end_time||
|GameId||

|Team* Field|Description|
|---|---|
|layout||
|score||
|session||
|shots||
|name||

### Upload training data
after you have done any transformation to the data you will need to upload the data to the appropriate data bucket. Talk with your opperations team to get the data bucket for your group

## Push and Commit
When you have a finish write new code to train and eploy your model commit the changes to your git branch and tell the opperations team to merge in your changes and deploy your code. 

```shell
git commit -a -m "I am adding these features <these features>"
git push -u origin $(git branch-name)
```